In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip
!wget https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip

--2021-03-12 07:46:42--  https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip.6’

horse-or-human.zip. 100%[===================>] 142.65M   163MB/s    in 0.9s    

2021-03-12 07:46:43 (163 MB/s) - ‘horse-or-human.zip.6’ saved [149574867/149574867]

--2021-03-12 07:46:43--  https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [

In [ ]:
import zipfile
import tensorflow as tf
from google.colab import files
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten,  Dense, Flatten, Dropout

In [ ]:
zip_validation_horse_or_human = zipfile.ZipFile('/content/validation-horse-or-human.zip', 'r')
zip_validation_horse_or_human.extractall('validation-horse-or-human')
zip_validation_horse_or_human.close()

zip_horse_or_human = zipfile.ZipFile('/content/horse-or-human.zip', 'r')
zip_horse_or_human.extractall('horse-or-human')
zip_horse_or_human.close()

In [ ]:
batch_size=100
image_size=(300, 300)
train_dataset = image_dataset_from_directory('horse-or-human', label_mode='int', seed=42, 
                                       subset='training',
                                       validation_split=0.2, 
                                       batch_size=batch_size, 
                                       image_size=image_size)

validation_dataset = image_dataset_from_directory('validation-horse-or-human', label_mode='int', seed=42,
                                             subset='validation',
                                             validation_split=0.2,
                                             batch_size=batch_size,
                                             image_size=image_size)

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)   

Found 1027 files belonging to 2 classes.
Using 822 files for training.
Found 256 files belonging to 2 classes.
Using 51 files for validation.


In [ ]:
input = Input(shape=(300, 300,3)) 

# Encoder
conv1_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input)
pool1 = MaxPooling2D((2, 2), padding='same')(conv1_1)
conv1_2 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D((2, 2), padding='same')(conv1_2)
conv1_3 = Conv2D(8, (3, 3), activation='relu', padding='same')(pool2)
encoder = MaxPooling2D((2, 2), padding='same')(conv1_3)


# Decoder
conv2_1 = Conv2D(8, (3, 3), activation='relu', padding='same')(encoder)
up1 = UpSampling2D((2, 2))(conv2_1)
conv2_2 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1)
up2 = UpSampling2D((2, 2))(conv2_2)
conv2_3 = Conv2D(32, (3, 3), activation='relu')(up2)
up3 = UpSampling2D((2, 2))(conv2_3)
decoder = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(up3)

tmp = Flatten()(decoder)
dence_1 = Dense(1024, activation='relu')(tmp)
dropout_1 = Dropout(0.2)(dence_1)
dence_2 = Dense(256, activation='relu')(dropout_1)
dropout_2 = Dropout(0.2)(dence_2)
output = Dense(2, activation='softmax')(dropout_2)

autoencoder = Model(inputs=input, outputs=output)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

autoencoder.summary()

# Обучаем модель
history = autoencoder.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=5,
                    verbose=1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 16)      4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 38, 38, 8)         0     

In [ ]:
autoencoder.save("horse_or_human.h5")

In [ ]:
files.download("horse_or_human.h5")